In [1]:
from utils.data_assistant_utils import *

### Get the accession number using openAI

---

In [2]:
accession = get_accession('You are an expert in the NCBI accession number formatting.',
                          'What is the NCBI accession number for the nucleotide sequence of the ACE2 receptor in humans?')

#### Print accession number to make sure that it looks valid

In [3]:
print(accession)

NM_021804


### Get the SeqRecord object based on that accession number

---

In [4]:
record, _ = fetch_sequence(accession, email='orogers@wesleyan.edu')

The sequence in GB format for ID NM_021804.3:
GGCACTCATACATACACTCTGGCAATGAGGACACTGAGCTCGCTTCTGAA

Homo sapiens angiotensin converting enzyme 2 (ACE2), transcript variant 2, mRNA


ValueError: too many values to unpack (expected 2)

In [5]:
sequence = record.seq

NameError: name 'record' is not defined

In [6]:
sequence

NameError: name 'sequence' is not defined

### Perform a Nucleotide BLAST

---

In [7]:
handle = nucleotide_blast(sequence)

NameError: name 'sequence' is not defined

#### Process the results handle

In [11]:
processed_stream = process_stream(handle, save=False, save_name='test')

NameError: name 'handle' is not defined

# Embed the sequences and their summaries using openAIembeddings

---

In [2]:
# Load previously generated BLAST results
df = pd.read_csv('ACE2_28Apr.csv')

In [4]:
print(df)

                                                title  length  e_value  \
0   gi|1700998533|ref|NM_021804.3| Homo sapiens an...    3596      0.0   
1   gi|66863987|dbj|AB193259.1| Homo sapiens ace2 ...    3605      0.0   
2   gi|2493543526|ref|XM_055376642.1| PREDICTED: G...    3596      0.0   
3   gi|2694562693|ref|XM_063634561.1| PREDICTED: S...    3597      0.0   
4   gi|1743233336|ref|XM_003261084.3| PREDICTED: N...    3593      0.0   
5   gi|66863989|dbj|AB193260.1| Homo sapiens ace2 ...    3470      0.0   
6   gi|8650465|gb|AF241254.1|AF241254 Homo sapiens...    3405      0.0   
7   gi|2490809741|ref|XM_008974180.3| PREDICTED: P...    6570      0.0   
8   gi|1700998531|ref|NM_001371415.1| Homo sapiens...    3339      0.0   
9   gi|37747876|gb|BC059378.1| Homo sapiens cDNA c...    3375      0.0   
10  gi|2697861032|ref|XM_016942979.3| PREDICTED: P...    3540      0.0   
11  gi|2695108862|ref|XM_063702989.1| PREDICTED: G...    3477      0.0   
12  gi|2695108865|ref|XM_019019204.3| 

In [18]:
from langchain_openai import OpenAIEmbeddings

In [19]:
embed = OpenAIEmbeddings(
    model='text-embedding-ada-002'
)

In [20]:
print(all_data['summary'][0])

REVIEWED REFSEQ: This record has been curated by NCBI staff. The
reference sequence was derived from AC097625.11 and AB193259.1.
On Jul 11, 2019 this sequence version replaced NM_021804.2.
Summary: The protein encoded by this gene belongs to the
angiotensin-converting enzyme family of dipeptidyl
carboxydipeptidases and has considerable homology to human
angiotensin 1 converting enzyme. This secreted protein catalyzes
the cleavage of angiotensin I into angiotensin 1-9, and angiotensin
II into the vasodilator angiotensin 1-7. ACE2 is known to be
expressed in various human organs, and its organ- and cell-specific
expression suggests that it may play a role in the regulation of
cardiovascular and renal function, as well as fertility. In
addition, the encoded protein is a functional receptor for the
spike glycoprotein of the human coronavirus HCoV-NL63 and the human
severe acute respiratory syndrome coronaviruses, SARS-CoV and
SARS-CoV-2, the latter is the causative agent of coronavirus
dis

In [ ]:
# DNABERT

In [21]:
print(all_data)

             title                                           sequence  \
0      NM_021804.3  (G, G, C, A, C, T, C, A, T, A, C, A, T, A, C, ...   
1       AB193259.1  (A, C, T, C, A, T, A, C, A, T, A, C, A, C, T, ...   
2   XM_055376642.1  (G, G, C, A, C, T, C, A, T, A, C, A, T, A, C, ...   
3   XM_063634561.1  (G, G, C, A, C, T, C, A, T, A, C, A, T, A, C, ...   
4   XM_003261084.3  (G, G, C, A, C, T, C, A, T, A, C, A, T, A, C, ...   
5       AB193260.1  (T, A, T, T, G, G, C, T, C, A, G, C, A, G, A, ...   
6       AF241254.1  (C, G, C, C, C, A, A, C, C, C, A, A, G, T, T, ...   
7   XM_008974180.3  (A, C, T, G, T, G, T, T, C, T, T, C, T, T, C, ...   
8   NM_001371415.1  (A, G, T, C, T, A, G, G, G, A, A, A, G, T, C, ...   
9       BC059378.1  (A, G, T, C, T, A, G, G, G, A, A, A, G, T, C, ...   
10  XM_016942979.3  (A, A, C, A, T, T, A, T, C, T, T, C, A, T, G, ...   
11  XM_063702989.1  (C, T, C, T, C, C, A, G, G, A, T, G, A, A, C, ...   
12  XM_019019204.3  (A, T, G, A, T, C, C, C, T, A, 

RuntimeError: Failed to import transformers.models.auto.tokenization_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)